In [23]:
import pandas as pd
import numpy as np
import pyhgvs as hgvs
import pyhgvs.utils as hgvs_utils
from pyfaidx import Fasta

## data source
html_file: the .html file downloaded from the UMD database

transcript: the transcript number for the gene

In [25]:
html_file = 'D:/CODE/BioData/UMD_gene/DMD/DMD.html'
TRANSCRIPT = 'NM_004006'
GENOME = 'D:/CODE/BioData/hg38/hg38.fa'
REF_GENE = 'D:/CODE/hgvs/pyhgvs/data/hg38/refGene.txt'

CONVERT = True
FILT_SNP = True
OUTPUT_PATH = './result.csv'

# Initialization
Initialize hgvs convertor

In [22]:
# Read genome sequence using pyfaidx.
genome = Fasta(GENOME)

# Read RefSeq transcripts into a python dict.
# with open('D:/CODE/hgvs/pyhgvs/data/genes.refGene') as infile:
with open(REF_GENE) as infile:
    transcripts = hgvs_utils.read_transcripts(infile)

# Provide a callback for fetching a transcript by its name.
def get_transcript(name):
    return transcripts.get(name)

UMD_columns = ['Protein nomenclature', 'cDNA Nomenclature', 'Exon', 'Codon',
       'Structure', 'HCD', 'Rearrangement', 'Mutation type',
       'Mutational event', '# records']

def task_convert(table, target_path):
    res_table = pd.DataFrame(
        columns=['chrom', 'offset', 'ref', 'alt'] + UMD_columns, index=None)
    cnt = 0
    for idx, row in table.iterrows():
        try:
            ts = TRANSCRIPT
            cmark = row['cDNA Nomenclature']
            # print(ts + ':' + str(cmark))
            chrom, offset, ref, alt = hgvs.parse_hgvs_name(
                ts + ':' + str(cmark), genome, get_transcript=get_transcript, normalize=True
            )
            # print(chrom,offset,ref,alt)
            # to make output file not too large
            if (len(ref)>200) or (len(alt)>200):
                continue
            if FILT_SNP and ( (len(ref) > 1) or (len(alt) > 1)):
                continue
            if ref == alt:
                continue
            feature_dict = {}
            for co in UMD_columns:
                feature_dict[co] = row[co]
            res_table.loc[cnt] = pd.Series({'chrom': chrom, 'offset': offset,
                                            'ref': ref, 'alt': alt,
                                            **feature_dict})
            cnt += 1
        except Exception as e:
            print(e)
    res_table.to_csv(target_path, sep='\t')
    return

# Load data

In [4]:
df = pd.read_html(html_file)
df = pd.DataFrame(df[0])
df.head()

     Protein nomenclature cDNA Nomenclature Exon   Codon  \
0                 p.Met1?       c.-648_0dup    1     1.0   
1                     NaN               NaN  NaN     NaN   
2                 p.Met1?      c.1_11055del    1     1.0   
3                     NaN               NaN  NaN     NaN   
4                 p.Met1?       c.1_1992del    1     1.0   
...                   ...               ...  ...     ...   
1709                  NaN               NaN  NaN     NaN   
1710           p.Gln3589X        c.10765C>T   75  3589.0   
1711                  NaN               NaN  NaN     NaN   
1712           p.Ser3637X        c.10910C>A   76  3637.0   
1713                  NaN               NaN  NaN     NaN   

                    Structure  HCD  \
0         Dp427m unique N-Ter  NaN   
1                         NaN  NaN   
2         Dp427m unique N-Ter  NaN   
3                         NaN  NaN   
4         Dp427m unique N-Ter  NaN   
...                       ...  ...   
1709         

In [5]:
df = df.dropna(subset=['Protein nomenclature'],axis=0)
columns = ['Protein nomenclature', 'cDNA Nomenclature', 'Exon', 'Codon',
       'Structure', 'HCD', 'Rearrangement', 'Mutation type',
       'Mutational event', '# records']
df = df[columns]
print('data size:{}'.format(len(df)))

749


In [26]:
if CONVERT:
    task_convert(df,OUTPUT_PATH)
else:
    df.to_csv()

Invalid HGVS cDNA allele "[1768T>C; 1769T>G]"
Invalid HGVS cDNA allele "[6766C>T; 6767T>A]"
